# Ejercicio 4.5

Con los mismos datos que el ejercicio nº 4.2, se pide:
* Realizar el triangulo de pagos por periodos mensuales, trimestrales y anuales, utilizando tablas pivotantes.

In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('Siniestros_2013_2018.csv', sep=';')

#Posibles freq: https://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
# Y=Anual, Q=Trimestral
freq='Y'

data['Fec_ocu'] = pd.to_datetime(data['Fec_ocu'], format='%d-%m-%Y')
data['Fec_pago'] = pd.to_datetime(data['Fec_pago'], format='%d-%m-%Y')
data['Fec_ocu'] = data['Fec_ocu'].dt.to_period(freq=freq)
data['Fec_pago'] = data['Fec_pago'].dt.to_period(freq=freq)

data.head()

,Siniestro,Poliza,Fec_ocu,Fec_com,Fec_pago,Importe
0,1,31082,2016,21-09-2016,2017,83
1,2,61275,2017,03-01-2018,2018,71
2,3,72257,2013,25-06-2013,2013,115
3,4,73111,2013,05-10-2013,2018,67
4,5,84755,2017,12-07-2017,2017,122


NOTA: En versiones de Pandas anteriores a 0.24.2 es necesario pasar a entero los campos para restarlos.
En este caso, utilizamos la librería datatime incluida en pandas con el alias __dt__

In [9]:
data['Dev'] = data['Fec_pago'].dt.year - data['Fec_ocu'].dt.year
data.head()

,Siniestro,Poliza,Fec_ocu,Fec_com,Fec_pago,Importe,Dev
0,1,31082,2016,21-09-2016,2017,83,1
1,2,61275,2017,03-01-2018,2018,71,1
2,3,72257,2013,25-06-2013,2013,115,0
3,4,73111,2013,05-10-2013,2018,67,5
4,5,84755,2017,12-07-2017,2017,122,0


In [11]:
datagg = data.groupby(['Fec_ocu','Dev']).agg({'Importe':'sum'}).reset_index()
datagg.head()

,Fec_ocu,Dev,Importe
0,2013,0,118066
1,2013,1,25877
2,2013,2,12392
3,2013,3,9743
4,2013,4,8599


In [12]:
triangulo = datagg.pivot(index='Fec_ocu', columns='Dev', values='Importe')
triangulo

Dev,0,1,2,3,4,5
Fec_ocu,,,,,,
2013,118066.0,25877.0,12392.0,9743.0,8599.0,8977.0
2014,119961.0,26543.0,13813.0,11509.0,10339.0,NaN
2015,108140.0,27975.0,18551.0,14250.0,NaN,NaN
2016,107660.0,34826.0,19814.0,NaN,NaN,NaN
2017,112042.0,51641.0,NaN,NaN,NaN,NaN
2018,133511.0,NaN,NaN,NaN,NaN,NaN


* Una vez calculado, exportar a un fichero Excel el triangulo anual de pagos acumulado.

In [8]:
acumulado = triangulo.cumsum(axis=1)
acumulado.to_excel('acumulado.xlsx')